# ARIMA

Now that we have analyzed the time series, we now must be able to forecast or predict future values based on previous values of the time series. One such model we can use is the ARIMA model, or the AutoRegressive Integrated Moving Average, a univariate time series forecasting model that only considers time as the independent variable. An ARIMA model can be characterized by 3 terms, $p, d, q$, where $p$ is the order of the autoregressive term, $q$ is the order of the moving average terms, and $d$ is the degree of differencing required to make the time series stationary. If the time series has seasonal patterns, then we must add seasonal terms to the model, and thus we can SARIMA, or Seasonal ARIMA. 

The autoregressive term, or $p$, refers to the number of lags of the target variable $y$ to be used as predictors, and the moving average term, or $q$, refers to the number of lagged forecastng errors of the autoregressive model that go into the ARIMA model. It follows that the ARIMA model is a combination of two other models, the autoregressive model(AR) and the Moving Average model(MA). The AR model is defined with the following formula: $$ Y_t = \mu + \beta_1 Y_{t-1} + \beta_2Y_{t-2}+ \dots + \beta_pY_{t-p} + \epsilon_t$$ where $\mu$ is the mean of the series, $Y_t$ is a function in terms of the lags of up to lag $p$, $\beta_i$ refers to the coefficient of $Y_{t-i}$, which the model estimates, or learns, and $\epsilon_t$ refers to the error term, the difference between the actual value $Y_t$ and the predicted value $\hat{Y_t}, \epsilon_t = Y_t - \hat{Y_t}$. The MA model is defined similarily: $$Y_{t} = \mu + \epsilon_t+ \phi_1\epsilon_{t-1}+\dots+\phi_q\epsilon_{t-q}$$ where the error terms $\epsilon_{t-i}$ refers to the error terms of the autoregressive model respective to the lagged term $Y_{t-i}$, and $\phi_i$ is the coefficients the model is predicting. The error term $\epsilon_t$ is obtained from the following autoregressive model: $$ Y_t = \mu + \beta_1Y_{t-1} + \dots + \beta_tY_0 + \epsilon_t$$

All together, the ARIMA model is a function $$Y_t = \mu + \beta_1Y_{t-1} + \dots + \beta_pY_{t-p} + \epsilon_t + \phi_1\epsilon_{t-1} + \dots + \phi_q\epsilon_{t-q}$$

Now, it is our job to find the values $p, q$, and $d$ so that our ARIMA model is efficient. 

### Finding order of differencing $d$

To find the degree of differencing $d$, it follows that we must interpret the autocorrelation function for each degree of differencing. If we see that the autocorrelations are positive and statistically significant for many number of lags, then the series need more degrees of differencing, as previous lags correlate significantly with the current value. On the other hand, if the autocorrelation for lag 1 is negative, then the series might be over-differenced, and a model with a lesser degree of differencing will probably work better. 

If we want to quantify the results on finding $d$, then we can use the ADFuller test and the KPSS test and analyze their p values. Thus, we can apply $d+1$ differencing to the time series and check the statistical tests until the tests agree that the series is stationary.

In [ ]:
# Graph the time series and its autocorrelation, then use ADFuller and KPSS to analyse the stationarity of the time series

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf

fig, axes = plt.subplots(3, 2,figsize=(12, 12))
axes[0, 0].plot(df['value'])
axes[1, 0].plot(df['value'].diff())
axes[2, 0].plot(df['value'].diff().diff())
plot_acf(df['value'].dropna(), ax=axes[0, 1])
plot_acf(df['value'].diff().dropna(), ax=axes[1, 1])
plot_acf(df['value'].diff().diff().dropna(), ax=axes[2, 1])
plt.show()

# d=0
result = adfuller(df['value'])
print(f'ADF Statistic of d=0: {result[0]}')
print(f'p-value: {result[1]}')
result = kpss(df['value'])
print(f'KPSS Statistic of d=0: {result[0]}')
print(f'p-value: {result[1]}')

# d=1
result = adfuller(df['value'].diff().dropna())
print(f'ADF Statistic of d=1: {result[0]}')
print(f'p-value: {result[1]}')
result = kpss(df['value'].diff().dropna())
print(f'KPSS Statistic of d=1: {result[0]}')
print(f'p-value: {result[1]}')

# d=2
result = adfuller(df['value'].diff().diff().dropna())
print(f'ADF Statistic of d=2: {result[0]}')
print(f'p-value: {result[1]}')
result = kpss(df['value'].diff().diff().dropna())
print(f'KPSS Statistic of d=2: {result[0]}')
print(f'p-value: {result[1]}')

### Finding order of AR term $p$

To find the AR term of the ARIMA model, it follows that we will inspect the partial autocorrelation function(PACF). The PACF is different from the ACF in the sense that it only considers the direct correlation between the current value and its lagged value, disregarding the intermediate correlations existant between both points. In the AR equation, it follows that the partial autocorrelation between a value and its lagged value is represented by the coefficient of that lag. For example, if we have the model equation of AR(3) as: $$Y_t = \mu + \beta_1 Y_{t-1} + \beta_2Y_{t-2} + \beta_3Y_{t-3} + \epsilon_t$$ then it follows that the partial autocorrelation of $Y_{t-3}$ is $\beta_3$. The AR equation uses the PACF because mathematically it is better to only consider the direct correlations between points and their lagged version of themselves, rather than factoring in the many indirect correlations in the equation.

Now, to find the $p$ value, the number of $Y_i$ terms we include in the equation, we must look at the PACF graph with respect to the degree of differencing we have chosen in the previous step, $d$. Then, we count the number of consecutive statistically significant points after lag 0; choosing points inside the significant interval is unnecessary and might cause overfitting.

In [ ]:
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf

# Plot time series with d=1 and pacf with d=1 of time series
plt.rcParams.update({'figure.figsize':(12, 6), 'figure.dpi':120})
fig, axes = plt.subplots(1, 2)
axes[0].plot(df['flow'].diff().dropna())
plot_pacf(df['flow'].diff().dropna(), ax=axes[1], lags=24*365)
plt.show()

### Finding order of MA term $q$

To find the MA term of the ARIMA model, it follows that we will look at the autocorrelation function based on the degree of differencing $d$, and find the number of consecutive statistically significant points after lag 0, which corresponds to the value $q$. Unlike the autoregressive model, it follows that the moving average model factors in the previous error terms of the time series and thus calculates the next value based on those error terms. 

### Building ARIMA Model

Like all machine learning models, to test a model, we must be able to split the data into training data and validation data. However, unlike machine learning models, it follows that the order must stay intact, as spliting the data by taking random points will make the ARIMA model ineffective. Thus, using normal cross-validation techniques in standard machine learning algorithms, such as k-fold cross-validation, is not possible in time series data, as we cannot use future data to predict past data. Therefore, we will split the time series based on time; we split the data into past data as our training data and future data as our validation data.

In [1]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()
TimeSeriesSplit(max_train_size=None, n_splits=5)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(df['value'], order=(2, 1, 2))
model_fit = model.fit()
model_fit.summary()

From the summary it follows that we can get the coefficients as well as their p values. We can use those p values to make some readjustments to the parameters of our ARIMA model.

We can also automate the ARIMA process in order to get the results quicker by using the pmdarima python library.

In [ ]:
import pmdarima as pm
auto_arima = pm.auto_arima(X_train)
auto_pred = auto_arima.predict(n_periods=len(val_X))

### Accuracy Metrics for ARIMA forecasting

When evaluating the performance of a time series model such as ARIMA

To compute the accuracy of our ARIMA model, we can use the mean absolute percentage error, correlation between the actual and the forecast values, and min-max error, so that our error values can be comparable to other data with different scales. If we only need to compare the error values with different models on the same dataset, then we can simply use the mean absolute error.